In [1]:
# Imports
from fastai.tabular.all import *
import pandas as pd

/home/francesc/anaconda3/envs/fraud-card-transactions/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370151529/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
df = pd.read_csv("data/clean_fraud.csv")
df.head()

,credit_card_num,shop,category,amount,gender,lat,long,city_pop,job,merch_lat,merch_long,is_fraud,age
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,0,31
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,0,41
2,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,Nature conservation officer,43.150704,-112.154481,0,57
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,0,52
4,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,0,33


In [3]:
tp = TabularPandas(df, procs=[Categorify],
                   cat_names = ['shop','category', 'gender','job'],
                   cont_names = ['credit_card_num', 'amount','lat','long','city_pop','merch_lat','merch_long','age'],
                   y_names='is_fraud',
                   y_block = CategoryBlock,
                   splits=RandomSplitter(valid_pct=0.2)(range_of(df)))

In [5]:
dls = tp.dataloaders(bs=64)
dls.show_batch()

,shop,category,gender,job,credit_card_num,amount,lat,long,city_pop,merch_lat,merch_long,age,is_fraud
0,fraud_Hamill-D'Amore,health_fitness,M,Arts development officer,2.252055e+15,75.269997,44.643700,-90.103104,828.0,45.605183,-89.668129,28.0,0
1,fraud_Kerluke PLC,misc_net,M,Hydrographic surveyor,2.450829e+15,5.960000,45.604000,-94.159103,16163.0,45.832767,-93.616646,34.0,0
2,fraud_Donnelly PLC,misc_net,M,Retail banker,4.671727e+15,45.110001,40.861801,-85.606697,2304.0,40.434429,-85.610580,64.0,0
3,"fraud_Morissette, Weber and Wiegand",grocery_net,F,"Scientist, audiological",4.129768e+15,37.959999,47.355099,-96.797997,771.0,47.456280,-95.896996,47.0,0
4,fraud_Koepp-Witting,grocery_pos,F,Tourism officer,6.763730e+11,134.779999,39.756199,-98.446198,242.0,39.649521,-98.428329,33.0,0
5,fraud_Kuhn LLC,shopping_pos,F,Statistician,3.462732e+14,3.390000,37.741798,-80.462601,4575.0,37.783951,-81.237137,28.0,0
6,fraud_Haley Group,misc_pos,F,Firefighter,3.521417e+15,15.280000,39.848999,-86.272003,910148.0,40.242340,-85.568886,45.0,0
7,fraud_Bogisich Inc,grocery_pos,F,"Teacher, primary school",2.131613e+14,73.650002,43.039700,-77.687103,10256.0,42.560970,-77.029739,52.0,0
8,fraud_Lind-Buckridge,entertainment,M,Historic buildings inspector/conservation officer,4.536997e+18,1.800000,35.986599,-106.065399,18408.0,36.392296,-106.747360,47.0,0
9,fraud_Koepp-Witting,grocery_pos,M,Volunteer coordinator,2.720012e+15,95.000000,43.352600,-102.541100,1126.0,42.764069,-101.569565,40.0,0


In [6]:
learn = tabular_learner(dls, metrics=accuracy)

In [7]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.012690,0.014435,0.995816,05:47


In [8]:
learn.show_results()

,shop,category,gender,job,credit_card_num,amount,lat,long,city_pop,merch_lat,merch_long,age,is_fraud,is_fraud_pred
0,336.0,14.0,2.0,323.0,3.502089e+15,2400.070068,39.029800,-77.079300,19054.0,38.260746,-77.252625,43.0,0.0,0.0
1,40.0,8.0,1.0,338.0,3.462083e+14,191.050003,40.710499,-74.016296,1577385.0,40.769569,-74.377274,59.0,0.0,0.0
2,134.0,1.0,2.0,409.0,3.585053e+15,20.670000,42.854599,-75.660500,176.0,43.627045,-75.764313,57.0,0.0,0.0
3,93.0,3.0,1.0,61.0,3.596217e+15,45.400002,40.862598,-91.953400,888.0,41.261658,-92.701187,31.0,0.0,0.0
4,446.0,6.0,2.0,135.0,3.712844e+14,5.710000,33.546600,-101.887100,254282.0,32.910355,-102.046616,67.0,0.0,0.0
5,130.0,2.0,2.0,324.0,3.741252e+14,57.470001,38.975700,-77.028198,601723.0,39.331326,-77.320221,50.0,0.0,0.0
6,265.0,8.0,1.0,457.0,4.629452e+15,7.620000,30.941601,-83.357399,10295.0,31.538189,-82.684280,22.0,0.0,0.0
7,115.0,11.0,1.0,310.0,5.020128e+11,13.690000,42.631500,-75.186600,165.0,43.322948,-76.123512,75.0,0.0,0.0
8,455.0,13.0,1.0,347.0,4.560395e+18,7.130000,39.047001,-122.932800,11256.0,38.698620,-122.333206,47.0,0.0,0.0
